In [66]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")

import sys


annualization_factor = 12 


file_path = "midterm_2_data.xlsx"  # assuming the file is in the directory
excel_file = pd.ExcelFile(file_path)
sheet_names = excel_file.sheet_names
sheet_names

['sector excess returns', 'factors excess returns']

In [67]:
factors = pd.read_excel("midterm_2_data.xlsx", sheet_name = 'factors excess returns')
factors = factors.set_index('date')
factors.head()

,MKT,HML,RMW,UMD
date,,,,
1980-01-01,0.0551,0.0175,-0.0170,0.0755
1980-02-01,-0.0122,0.0061,0.0004,0.0788
1980-03-01,-0.1290,-0.0101,0.0146,-0.0955
1980-04-01,0.0397,0.0106,-0.0210,-0.0043
1980-05-01,0.0526,0.0038,0.0034,-0.0112


In [68]:
sectors = pd.read_excel("midterm_2_data.xlsx", sheet_name = 'sector excess returns')
sectors = sectors.set_index('date')
sectors.head()

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,-0.0076,0.0285,0.0084,0.1009,-0.0143,0.1002,0.0362,0.0323,0.0048,0.0059,...,0.0158,0.0875,0.0465,-0.0126,0.0430,-0.0283,0.0258,0.0768,0.0308,0.0669
1980-02-01,0.0105,-0.0608,-0.0966,-0.0322,-0.0569,-0.0323,-0.0521,-0.0800,-0.0555,-0.0167,...,-0.0079,-0.0541,-0.0346,-0.0639,-0.0652,-0.0854,-0.0959,-0.0347,-0.0282,-0.0274
1980-03-01,-0.2224,-0.1119,-0.0167,-0.1469,-0.0193,-0.1271,-0.0826,-0.1237,-0.0566,-0.0668,...,-0.0819,-0.1509,-0.1098,-0.0906,-0.1449,-0.0560,-0.0880,-0.2451,-0.1254,-0.1726
1980-04-01,0.0449,0.0766,0.0232,0.0321,0.0830,-0.0529,0.0783,0.0153,0.0304,0.0115,...,0.0420,-0.0103,-0.0312,0.0353,0.0542,0.0728,0.0530,0.0977,0.0447,0.0769
1980-05-01,0.0632,0.0793,0.0457,0.0863,0.0815,0.0509,0.0324,0.0886,0.0560,0.0098,...,0.0564,0.1063,0.1142,0.0877,0.1134,0.0578,0.0557,0.0915,0.0844,0.0685


In [107]:
import numpy as np 

def portfolio_returns_calc(data, weights_df): 
    return pd.DataFrame(data @ (weights_df.values)) 
    
# returns is the main returns not annualized 
import numpy as np
import pandas as pd
from typing import Union

def tangency_weights(
    returns: pd.DataFrame,
    dropna: bool = True,
    scale_cov: float = 1,
    name: str = "Tangency",
    target_ret_rescale_weights: Union[None, float] = None,
    annual_factor: int = 12,
    expected_returns: Union[None, pd.Series] = None,
    expected_returns_already_annualized: bool = False
) -> pd.DataFrame:
    # Drop missing data
    if dropna:
        returns = returns.dropna()

    # Calculate regularized covariance matrix
    covmat_full = returns.cov()
    covmat_diag = np.diag(np.diag(covmat_full))
    covmat = scale_cov * covmat_full + (1 - scale_cov) * covmat_diag
    covmat *= annual_factor  # Annualize covariance matrix

    # Inverse of the covariance matrix
    covmat_inv = np.linalg.pinv(covmat)

    # Calculate expected returns
    if expected_returns is not None:
        mu = expected_returns.copy()
        if not expected_returns_already_annualized:
            mu *= annual_factor
    else:
        mu = returns.mean() * annual_factor  # Annualize mean returns

    # Tangency portfolio weights (unscaled)
    weights = covmat_inv @ mu
    weights /= weights.sum()  # Normalize weights to sum to 1

    # Create DataFrame for weights
    tangency_weights_df = pd.DataFrame(weights, index=returns.columns, columns=[f"{name} Weights"])

    # Rescale weights to target return (if specified)
    if target_ret_rescale_weights is not None:
        port_return = (weights @ mu)  # Portfolio return with current weights
        scaler = target_ret_rescale_weights / port_return
        tangency_weights_df *= scaler  # Rescale weights

    return tangency_weights_df


In [73]:
train = sectors.loc['2018':'2022']
test = sectors.loc['2023':]
train_f = factors.loc['2018':'2022']
test_f = factors.loc['2023':]
train_f

,MKT,HML,RMW,UMD
date,,,,
2018-01-01,0.0557,-0.0128,-0.0075,0.0405
2018-02-01,-0.0365,-0.0104,0.0048,0.0355
2018-03-01,-0.0235,-0.0020,-0.0042,-0.0113
2018-04-01,0.0029,0.0054,-0.0244,0.0037
2018-05-01,0.0265,-0.0322,-0.0205,0.0387
2018-06-01,0.0048,-0.0233,0.0080,-0.0236
2018-07-01,0.0319,0.0045,0.0155,-0.0142
2018-08-01,0.0344,-0.0400,-0.0031,0.0528
2018-09-01,0.0006,-0.0171,0.0063,0.0004


In [74]:
train.shape, train_f.shape

((60, 49), (60, 4))

In [93]:
def factor_model(portfolios: pd.DataFrame, factors_used: list, factors_data: pd.DataFrame, name: str) -> pd.DataFrame: 
    
    summary = pd.DataFrame(index=portfolios.columns)

    for asset in portfolios.columns:
        asset_ret = portfolios[asset]
        
        x = factors_data[factors_used]
        model = sm.OLS(asset_ret, x).fit()
        for i in range(len(factors_used)):
            factor = factors_used[i]
            summary.loc[asset, f'{factor}'] = model.params[i]
    
    return summary 
    
summary = factor_model(train, list(train_f.columns), train_f, 'FF')     
summary.head()

,MKT,HML,RMW,UMD
Agric,0.832362,0.556541,-0.502093,0.038972
Food,0.524509,0.205452,0.309711,-0.003572
Soda,0.540240,0.179127,0.638443,0.013688
Beer,0.541267,0.022959,0.629678,-0.040541
Smoke,0.498185,0.443078,0.402220,-0.134015


In [97]:
factor_mean = train_f.mean()
er = summary.mul(factor_mean, axis=1).sum(axis=1)
er.head()

Agric    0.003655
Food     0.005454
Soda     0.007336
Beer     0.007137
Smoke    0.005143
dtype: float64

In [99]:
(er * annualization_factor).head()

Agric    0.043861
Food     0.065451
Soda     0.088035
Beer     0.085648
Smoke    0.061716
dtype: float64

In [105]:
regularized_weights_df.head()

,Regularized Weights
Agric,0.144090
Food,-0.069810
Soda,0.322680
Beer,0.107614
Smoke,-0.043393


In [112]:
tan_weights = tangency_weights(
    returns= train,
    dropna=True,
    scale_cov = 0.5,
    name = "Tangency",
    annual_factor=annualization_factor,
    expected_returns = er,
)
tan_weights.head()

,Tangency Weights
Agric,-0.030723
Food,0.015320
Soda,0.132944
Beer,0.079421
Smoke,0.012323


In [115]:
# use this allocation in the test 
returns_test = portfolio_returns_calc(test, tan_weights)

def portfolio_metrics_basic(returns: pd.DataFrame, annualization_factor: float) -> pd.DataFrame: 
    '''
    Returns Annualized ER, Vol, SR, 
    '''
    
    returns_df = pd.DataFrame(index = returns.columns) 
    mu = returns.mean() * annualization_factor
    vol = returns.std() * np.sqrt(annualization_factor)
    sr = mu / vol

    
    returns_df['Annnualized Return'] = mu.values 
    returns_df['Annnualized Vol'] = vol.values 
    returns_df['Annnualized SR'] = sr.values
    
    return returns_df

portfolio_metrics_basic(returns_test,annualization_factor)

,Annnualized Return,Annnualized Vol,Annnualized SR
0,0.181176,0.119549,1.515494


In [127]:
# construct tan portfolio. replace risk premia with sample average from train 
tan_weights = tangency_weights(
    returns= train,
    dropna=True,
    scale_cov = 0.5,
    name = "sample_avg",
)
returns_sample = portfolio_returns_calc(test, tan_weights)
portfolio_metrics_basic(returns_sample,annualization_factor)


,Annnualized Return,Annnualized Vol,Annnualized SR
0,0.176801,0.15301,1.155487


In [132]:
# build a tan portfolio using factors only 
factor_tan_weights = tangency_weights(
    returns= train_f,
    dropna=True,
    scale_cov = 0.5,
    name = "factors_tan",
    expected_returns = train_f.mean(),
)
returns_factors = portfolio_returns_calc(test_f, factor_tan_weights)
portfolio_metrics_basic(returns_factors,annualization_factor)


,Annnualized Return,Annnualized Vol,Annnualized SR
0,0.062376,0.058191,1.071918


# Long Run Returns

In [38]:
import numpy as np
factors_log = np.log(factor + 1)
factors_log

,MKT,HML,RMW,UMD
date,,,,
1980-01-01,0.053636,0.017349,-0.017146,0.072786
1980-02-01,-0.012275,0.006081,0.000400,0.075849
1980-03-01,-0.138113,-0.010151,0.014494,-0.100373
1980-04-01,0.038932,0.010544,-0.021224,-0.004309
1980-05-01,0.051263,0.003793,0.003394,-0.011263
...,...,...,...,...
2024-04-01,-0.047826,-0.005214,0.014692,-0.004209
2024-05-01,0.042485,-0.016739,0.029365,-0.000200
2024-06-01,0.027323,-0.033660,0.005087,0.008960


In [39]:
factors_log.mean(), factors_log.std() 

(MKT    0.006129
 HML    0.001647
 RMW    0.003628
 UMD    0.004175
 dtype: float64,
 MKT    0.045853
 HML    0.031691
 RMW    0.024125
 UMD    0.046313
 dtype: float64)

In [40]:
def portfolio_metrics_basic(returns: pd.DataFrame, annualization_factor: float) -> pd.DataFrame: 
    '''
    Returns Annualized ER, Vol, SR, 
    '''
    
    returns_df = pd.DataFrame(index = returns.columns) 
    mu = returns.mean() * annualization_factor
    vol = returns.std() * np.sqrt(annualization_factor)
    sr = mu / vol

    
    returns_df['Annnualized Return'] = mu.values 
    returns_df['Annnualized Vol'] = vol.values 
    returns_df['Annnualized SR'] = sr.values
    
    return returns_df
annualization_factor = 12
metrics = portfolio_metrics_basic(factors_log,annualization_factor)
metrics

,Annnualized Return,Annnualized Vol,Annnualized SR
MKT,0.073549,0.158841,0.463033
HML,0.019770,0.109782,0.180081
RMW,0.043540,0.083573,0.520979
UMD,0.050095,0.160433,0.312248


In [41]:
metrics_15.columns

Index(['Annnualized Return', 'Annnualized Vol', 'Annnualized SR'], dtype='object')

In [42]:
# since log returns  are iid, 
metrics_15 = metrics.copy() 
metrics_15['Annnualized Return'] = metrics_15['Annnualized Return'] * 15
metrics_15['Annnualized Vol'] = metrics_15['Annnualized Vol'] * np.sqrt(15)
metrics_15['Annnualized SR'] = metrics_15['Annnualized SR'] * np.sqrt(15)
metrics_15

,Annnualized Return,Annnualized Vol,Annnualized SR
MKT,1.103228,0.615188,1.793318
HML,0.296544,0.425182,0.697452
RMW,0.653096,0.323676,2.017743
UMD,0.751422,0.621353,1.209332


In [43]:
# probability that momentum fator has a negative mean excess return 

from scipy.stats import norm 

def prob(mu, sigma, h): 
    return norm.cdf(np.sqrt(h) * mu/sigma) 

# negative excess return (left side) 
prob_1_month = 1- prob(metrics.loc['UMD', 'Annnualized Return'], metrics.loc['UMD', 'Annnualized Vol'], 1/annualization_factor) 
prob_15_years = 1- prob(metrics.loc['UMD', 'Annnualized Return'], metrics.loc['UMD', 'Annnualized Vol'], 15) 
prob_1_month, prob_15_years

(0.4640886761424672, 0.1132677468366372)

In [49]:
log_spread = factors_log.loc[factors_log.index >= '2009']
log_spread = log_spread['UMD'] - log_spread['MKT']
log_spread_df = portfolio_metrics_basic(log_spread.to_frame('log_spread'),annualization_factor)
log_spread_df

,Annnualized Return,Annnualized Vol,Annnualized SR
log_spread,-0.154803,0.270968,-0.571297


In [52]:
# outperform > 0 
prob_1_month = prob(log_spread_df.loc['log_spread', 'Annnualized Return'], log_spread_df.loc['log_spread', 'Annnualized Vol'], 1/annualization_factor) 
prob_15_years =  prob(log_spread_df.loc['log_spread', 'Annnualized Return'], log_spread_df.loc['log_spread', 'Annnualized Vol'], 15) 
prob_1_month, prob_15_years

(0.4345038271448206, 0.013461859038858909)